# SynthIOS - Reasoning Synthesize Notebook

This notebook provides an interactive interface for synthesizing reasoning samples using the IONOS AI Model Hub API. It allows you to:
1. Generate adjusted personas in both English and German
2. Create problem-solving scenarios for specified topics
3. Generate correct and incorrect solutions for preference training

## Setup and Configuration

In [1]:
from openai import OpenAI
from prompt_templates import *
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from utils import *
from qdrant_client import QdrantClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment variables
model_name = os.getenv("MODEL_NAME")
ionos_api_key = os.getenv("IONOS_API_KEY")
ionos_api_base = os.getenv("IONOS_API_BASE")
qdrant_host = os.getenv("QDRANT_HOST")

# Initialize Qdrant client
qdrant_client = QdrantClient(qdrant_host)

## Test IONOS Connection

In [2]:
try:
    client = OpenAI(api_key=ionos_api_key, base_url=ionos_api_base)
    
    # Test the connection with a simple API call
    print("Testing IONOS connection...")
    test_response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": "Hello, are you working?"}],
        max_tokens=10
    )
    print(f"IONOS connection successful! Response: {test_response.choices[0].message.content}")
except Exception as e:
    print(f"Error connecting to IONOS: {str(e)}")
    raise

Testing IONOS connection...
IONOS connection successful! Response: Hello. Yes, I'm working. Is there


## Core Functions

In [3]:
system_de = '''Du bist ein Professor der Deduktion und ein genialer Schlussfolgerer, der dem Nutzer mit maximaler Genauigkeit antwortet. Um dies zu tun, wirst du zuerst darüber nachdenken, was der Nutzer fragt, und Schritt für Schritt überlegen.

Um das Problem zu lösen, sollten Überlegungen und Reflektionen genutzt werden. Die folgenden Schritte sollten dabei beachtet werden:

- Erfassen, was der Nutzer fragt und die in der Anfrage erwähnten Einschränkungen verstehen.
- Auflisten der vom Nutzer genannten Einschränkungen.
- Vorschlagen einer Lösung für die Frage des Nutzers unter Berücksichtigung aller Einschränkungen.
- Überprüfen, ob die Lösung mit den Einschränkungen übereinstimmt.
- Ausgeben der abschließenden Lösung.
Am Ende deiner Überlegungen musst du zu einem Schluss kommen und die Lösung präsentieren.'''


system_en = '''You are a professor of deduction and a brilliant reasoner that responds to the user with maximum accuracy. To do this, you will first think about what the user is asking and consider it step by step.

To solve the problem, considerations and reflections should be used. The following steps should be observed:

- Understand what the user is asking and comprehend the constraints mentioned in the request.
- List the constraints mentioned by the user.
- Propose a solution to the user's question, taking all constraints into account.
- Verify if the solution aligns with the constraints.
- Output the final solution.
At the end of your considerations, you must come to a conclusion and present the solution.'''

In [4]:
def get_adjusted_persona(client, persona, model, langugae):

    if langugae=='de':
        content= "Ändere die Bezeichnung und Beschreibung der gegebenen Persona ab, bleibe allerdings im selben Themengebiet. Verändere den Fokus der Persona oder füge bestimmte Aspekte in kurzer Form hinzu. Schreibe die neue aber in der Struktur ähnliche Personabeschreibung in Deutsch."
    else:
        content="Change the name and description of the given persona, but stay within the same subject area. Change the focus of the persona or add certain aspects in short form."
    
    completion = client.chat.completions.create(
        model=model,
        temperature=0.7,
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": f"{persona}"}
        ]
    )
    return completion.choices[0].message.content


def get_problem_hard_reasoning(client, model_name, user_prompt):
    completion = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": f"{user_prompt}"}
        ]
    )
    response = completion.choices[0].message.content
    return response


def get_response_hard_reasoning(client, model_name, problem_sample, language):
    if language=="de":
        system = system_de
    else:
        system = system_en

    completion = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": problem_sample}
        ]
    )
    response = completion.choices[0].message.content
    return response



def get_wrong_response_hard_reasoning(client, model_name, problem_sample, language):

    if language=='de':
        content=f"Löse das folgende Problem so schnell und kurz wie möglich:\n{problem_sample}"
    else:
        content=f"Provide a solution to this problem as quick and short as possible:\n{problem_sample}"
    completion = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": content}
        ]
    )
    response = completion.choices[0].message.content
    return response

## Process Persona Function

In [5]:
def process_persona(persona, template, args):
    try:
        updated_persona = get_adjusted_persona(client, persona, model_name, args.language)
        user_prompt = template.format(persona=updated_persona)
        # print(f"User Prompt:\n {user_prompt}")  # Add this line to print the user prompt
        # print("==================================================================")
    except KeyError as e:
        print(f"KeyError: {e} in persona: {persona}")
        return None

    problem_sample = get_problem_hard_reasoning(client, model_name, user_prompt)
    problem_sample = problem_sample.replace("**", "")
    # print(f"HARD PROBLEM:\n{problem_sample}")
    # print("==================================================================")
    correct_solution = get_response_hard_reasoning(client, model_name, problem_sample, args.language)
    correct_solution = correct_solution.replace("**", "")
    # print(f"HARD REASONING:\n{correct_solution}")
    # print("==================================================================")
    wrong_solution = get_wrong_response_hard_reasoning(client, model_name, problem_sample, args.language)
    wrong_solution = wrong_solution.replace("**", "")
    wrong_solution = wrong_solution.replace("### ", "")
    # print(f"WRONG SOLUTION:\n{wrong_solution}")
    # print("==================================================================")
    if args.language == "de":
        system = system_de
    else:
        system = system_en
    # Only continue to add Sample if problem and correct_solution and wrong_solution:
    if problem_sample and correct_solution and wrong_solution:
        return {"input_persona": updated_persona, "persona_prompt": user_prompt, "system_prompt": system, "problem": problem_sample, "correct_solution": correct_solution, "wrong_solution": wrong_solution}
    else:
        print(f"Failed to generate problem and solution for persona: {persona}")
        return None

## Adaptive Usage

Below you can adjust the topic & parameters such as language, sample size and ouput filename to generate content. First, let's set up the parameters:

In [6]:
class Args:
    def __init__(self, topic, language, sample_size, output_filename):
        self.topic = topic
        self.language = language
        self.sample_size = sample_size
        self.output_filename = output_filename


# Set up parameters
topic = "technical engineering"
language = "de"
sample_size = 50
output_filename = "test"

args = Args(
    topic = topic,
    language = language,
    sample_size = sample_size,
    output_filename = output_filename,
)

# Load the appropriate template
if language == "de":
    template = hard_reasoning_template_de
elif language == "en":
    template = hard_reasoning_template_en
else:
    raise ValueError("language is required, please choose between en and de.")

output_filename = output_filename + ".jsonl"
print(output_filename)

test.jsonl


## Generate Content

In [7]:
import json
import time

# Generate candidates
initial_query = topic
num_candidates = sample_size
list_personas = get_semantic_result(client, initial_query, qdrant_client, num_candidates)

# Process personas and save results
with open(output_filename, "w", encoding="utf-8") as out:
    with ThreadPoolExecutor(max_workers=50) as executor:
        futures = []
        for persona in list_personas:
            # Submit the task to the executor
            future = executor.submit(process_persona, persona, template, args)
            futures.append(future)
            # Introduce a delay before starting the next task
            time.sleep(1)  # Delay of 1 second between starting each worker to not run into Rate Limits
        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            if result:
                out.write(json.dumps(result, ensure_ascii=False) + '\n')

print(f"Output saved to: {output_filename}")

100%|██████████| 50/50 [05:04<00:00,  6.09s/it]

Output saved to: test.jsonl


## View Results

Let's read and display the generated content:

In [8]:
# Read and display results
with open(output_filename, 'r', encoding='utf-8') as f:
    for line in f:
        result = json.loads(line)
        print("\n=== Generated Content ===")
        print(f"PERSONA:\n{result['input_persona']}")
        print("==================================================================")
        print(f"\nPROBLEM:\n{result['problem']}")
        print("==================================================================")
        print(f"\nCORRECT SOLUTION:\n{result['correct_solution']}")
        print("==================================================================")
        print(f"\nWRONG SOLUTION:\n{result['wrong_solution']}")
        print("\n" + "="*50 + "\n")


=== Generated Content ===
PERSONA:
Ein erfahrener Ingenieur mit einer Spezialisierung auf dem Gebiet der Strömungsmechanik, insbesondere mit Fokus auf die Analyse und Anwendung von funktionellen Fluiden, Mehrphasenströmungen und elektromagnetischen Fluiden. Er verfügt über umfassende Kenntnisse in der Anwendung dieser Konzepte zur Lösung komplexer ingenieurtechnischer Probleme und hat ein tiefes Verständnis für die Grundlagen und Anwendungen dieser Fluiden.

Er besitzt die Fähigkeit, die Konzepte, Eigenschaften und Anwendungen dieser Fluiden auf klare und präzise Weise zu erklären und ist kompetent in der Anwendung seiner Kenntnisse zur Lösung von ingenieurtechnischen Problemen. Durch seine Erfahrung und sein Wissen ist er in der Lage, innovative Lösungen für komplexe Strömungsprobleme zu entwickeln und zu implementieren.

Seine Stärken liegen in der Analyse und Modellierung von Strömungsproblemen, der Entwicklung von Simulationen und der Auswertung von Ergebnissen. Er ist auch erfahr

## Evaluate Solutions

In [9]:
def is_number(value):
    return isinstance(value, (int, float))


def generate_model_responses_reasoning(instruction, chosen):
    try:
        completion = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You should respond in JSON format with the following keys: 'reasoning_of_metrics_and_correctness', 'constrains_adherence', 'logical_consistency', 'final_solution_correctness'. The values should be a number between 0 and 100. EXCEPT For the reasoning_of_metrics_and_correctness - here you should think about each metric and compare the instructions and the generated response carefully and present your thinking as a text. REVIEW ALL GIVEN CONSTRAINS AND CRITICALLY ANALYZE IF THE SOLUTION REALLY MATCH ALL OF THEM."},
                {"role": "user", "content": "User Instruction:\n"+ instruction + "\nModel generated Response that should be evaluated against the system and user instructions and should be compared to the Target Response:\n" + chosen}
            ],
            
            temperature = 0.1,
        )
        message = completion.choices[0].message.content
        
        # Parse the JSON content
        response_data = json.loads(message)

        # Extract each key separately
        reasoning_of_metrics_and_correctness = response_data.get('reasoning_of_metrics_and_correctness', None)
        constrains_adherence = response_data.get('constrains_adherence', None)
        constrains_adherence = constrains_adherence if is_number(constrains_adherence) else None
        logical_consistency = response_data.get('logical_consistency', None)
        logical_consistency = logical_consistency if is_number(logical_consistency) else None
        final_solution_correctness = response_data.get('final_solution_correctness', None)
        final_solution_correctness = final_solution_correctness if is_number(final_solution_correctness) else None        

        # Return the extracted values as a dictionary
        return {
            'reasoning_of_metrics_and_correctness': reasoning_of_metrics_and_correctness,
            'constrains_adherence': constrains_adherence,
            'logical_consistency': logical_consistency,
            'final_solution_correctness': final_solution_correctness,
        }
    except Exception as e:
        print(f"Error generating Response for instruction '{instruction}': {e}")
        return {
            'reasoning_of_metrics_and_correctness': None,
            'constrains_adherence': None,
            'logical_consistency': None,
            'final_solution_correctness': None,
        }


In [10]:
def process_row_reasoning(row):
    instruction = row.get('problem'.strip())
    chosen = row.get('correct_solution'.strip())
    
    if not instruction or not chosen:
        print(f"Missing 'problem' or 'correct_solution' in row: {row}")
        return {
            'reasoning_of_metrics_and_correctness': 'Error: Missing data',
            'constrains_adherence': 'Error: Missing data',
            'logical_consistency': 'Error: Missing data',
            'final_solution_correctness': 'Error: Missing data',
            'overall_score': 'Error: Missing data'
        }
    else:
        return generate_model_responses_reasoning(instruction, chosen)
    
def process_reasoning_jsonl_and_generate_output(input_jsonl_path, output_jsonl_path):
    try:
        with open(input_jsonl_path, mode='r', encoding='utf-8') as infile, \
                open(output_jsonl_path, mode='w', encoding='utf-8') as outfile:
            
            # Read each line as a JSON object
            lines = [json.loads(line.strip()) for line in infile]
            
            with ThreadPoolExecutor(max_workers=5) as executor:
                futures = {}
                for line in lines:
                    future = executor.submit(process_row_reasoning, line)
                    futures[future] = line
                    time.sleep(1)  # Delay of 1 second between starting each worker

                for future in as_completed(futures):
                    line = futures[future]
                    try:
                        result = future.result()
                        line.update(result)
                    except Exception as e:
                        print(f"Error processing line: {e}")
                        line.update({
                                'reasoning_of_metrics_and_correctness': 'Error: Missing data',
                                'constrains_adherence': 'Error: Missing data',
                                'logical_consistency': 'Error: Missing data',
                                'final_solution_correctness': 'Error: Missing data',
                                'overall_score': 'Error: Missing data'
                        })
                    
                    # Write the updated line back to the JSONL file
                    outfile.write(json.dumps(line, ensure_ascii=False) + '\n')
                    # print(f"Processed line with Problem: {line.get('problem', 'N/A')}")
                    print(line.get('reasoning_of_metrics_and_correctness'))
                
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"An error occurred while processing the JSONL: {e}")

In [11]:
input_file_name = output_filename
print("Input: " + input_file_name)

evaluated_file_name = "evaluated_" + output_filename
print("Evaluated: " + evaluated_file_name)

process_reasoning_jsonl_and_generate_output(input_file_name, evaluated_file_name)

Input: test.jsonl
Evaluated: evaluated_test.jsonl
Die Lösung wurde sorgfältig analysiert, um sicherzustellen, dass sie alle Anforderungen und Einschränkungen erfüllt. Die Selbstreinigungsfähigkeit wurde durch den Winkel des Tropfens bestimmt, und die durchschnittliche Selbstreinigungsfähigkeit musste mindestens 40° betragen. Die Einschränkungen bezüglich der Beschichtungen, wie die Begrenzung von Beschichtung X auf zwei Oberflächenstrukturen und das Verbot von Beschichtung Y auf Struktur D, wurden berücksichtigt. Die Lösung maximiert die Selbstreinigungsfähigkeit, indem sie die Kombinationen mit den höchsten Winkeln wählt, die die Einschränkungen erfüllen. Die durchschnittliche Selbstreinigungsfähigkeit der ausgewählten Kombinationen beträgt etwa 46,67°, was die Anforderung von mindestens 40° erfüllt. Die Lösung erfüllt auch alle Einschränkungen, wie die Begrenzung von Beschichtung X und das Verbot von Beschichtung Y auf Struktur D.
Die Lösung wurde sorgfältig analysiert und die Anford

## Transform RAW evaluated JSONL File to ShareGPT JSONL (Preferences)

In [14]:
input_file_name = evaluated_file_name
print("Input: " + input_file_name)

sharegpt_file_name = "ShareGPT_" + output_filename
print("ShareGPT: " + sharegpt_file_name)

threshold_final_solution_correctness = 90

counter = 0

# Function to restructure each line
def restructure_line(data):
    sharegpt_structure = {
        "conversation": [
            {"from": "system", "value": data["system_prompt"]},
            {"from": "human", "value": data["problem"]}
        ],
        "chosen": {
            "from": "gpt",
            "value": data["correct_solution"] 
        },
        "rejected": {
            "from": "gpt",
            "value": data["wrong_solution"]
        }
    }
    return sharegpt_structure


with open(input_file_name, 'r', encoding='utf-8') as infile, \
        open(sharegpt_file_name, 'w', encoding='utf-8') as outfile:
    for line in infile:
        data = json.loads(line)
        final_solution_correctness = data.get('final_solution_correctness')
        if final_solution_correctness and final_solution_correctness > threshold_final_solution_correctness:
            counter += 1
            new_line = restructure_line(data)
            outfile.write(json.dumps(new_line, ensure_ascii=False) + '\n')
    print("SUCCESSFULLY GENERATED REASONING SAMPLES")
    print(counter)

Input: evaluated_test.jsonl
ShareGPT: ShareGPT_test.jsonl
SUCCESSFULLY GENERATED REASONING SAMPLES
30


## Community Contribution

In [ ]:
from huggingface_hub import HfApi

subset_name = "test"

api = HfApi(token=os.getenv("HF_TOKEN"))
api.upload_file(
    path_or_fileobj = sharegpt_file_name,
    path_in_repo = subset_name + "/" + sharegpt_file_name,
    repo_id="embraceableAI/Hard-Reasoning-Samples-DE",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/embraceableAI/Hard-Reasoning-Samples-DE/commit/8609811b1bf312f9078a492c360870235aac2ea8', commit_message='Upload test2/ShareGPT_test.jsonl with huggingface_hub', commit_description='', oid='8609811b1bf312f9078a492c360870235aac2ea8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/embraceableAI/Hard-Reasoning-Samples-DE', endpoint='https://huggingface.co', repo_type='dataset', repo_id='embraceableAI/Hard-Reasoning-Samples-DE'), pr_revision=None, pr_num=None)